* 上证50 ——zssh000016——page544——content40892  √
* 沪深300——zssh000300——page420——content32672   √
* 中证1000——zssh000852——page41——content2589   √
* 中证500——zssh000905——page68——content4774  √
* 上证指数——zssh000001——page71370——content5617203
* 深证成指——zssz399001——page5740——content447793  √


* 上交所50ETF——sh510050——804——55591
* 沪深300ETF——sh510300——374——24403
* 中证500ETF——sh510500——106——7156
* 科创50ETF——sh588000——298——22449
* 豆粕ETF——sz159985——40——2766
* 黄金ETF——sh518880——57——4222

In [ ]:
import pandas as pd        #保存文件
import numpy as np
import requests            #获取网页
from lxml import etree     #解析文档
from faker import Factory# 生成不同的user-agent  ## 可能需要 pip install faker
import time
import random 
from multiprocessing.dummy import Pool as ThreadPool  # 线程池
import re
#from Crawler import crawler_url_list ## 封装函数

In [ ]:
stock = "sh518880"

In [ ]:
## 读取数据
#url_list = pd.read_csv(f"url_list_{stock}.csv",header=None)
url_list = pd.read_csv(f"url_list_{stock}_ETF.csv",header=None)
url_list = url_list.iloc[:,0].tolist()

In [ ]:
len(url_list)

In [ ]:
block = int(input("需要处理的block："))  ## 目前在 【 0 】 

In [ ]:
"""
5百多万数据过多，计划每50万为一个Block。
"""
block_nums = 500000

def get_block_list(url_list,block_nums, start=0):  ## 获取 分部分爬虫list
    url_nums = len(url_list)
    temp_index = np.arange(start,url_nums,block_nums).tolist()
    temp_index.append(url_nums)
    
    part_block_list = []
    for i in range(len(temp_index)-1):
        part_block_list.append(url_list[temp_index[i]:temp_index[i+1]])
    return part_block_list

url_list_block = get_block_list(url_list,block_nums, start=0)
print("总共有",len(url_list_block),"个block")

url_list = url_list_block[block]
print("当前block个内有",len(url_list_block),"个url")
print("*******当前为第",block+1,"个block*******")

In [ ]:
## 读取 IPpoor.text
IP_pool = []
file=open('IP_poor_0103.txt')
for f in file.read().splitlines():
    IP_pool.append(eval(f))
print(len(IP_pool))

In [ ]:
def text_deal(temp_text):  ## 文本处理函数
    str = ''
    temp_text1 = str.join(temp_text).strip()  ## 合并分散的文本
    temp_text1 = re.sub('[\r\n]', '', temp_text1).strip()
    temp_text2 = re.sub('[\xa0]', '', temp_text1).strip()  ## 清楚空格
    return temp_text2

In [ ]:
def get_guba_content(url): ## 获取 url里内容和日期
    ## 生成随机user-agent
    fc = Factory.create() 
    header = {'User-Agent': fc.user_agent()}
    ## 获取网页
    response = requests.get(url = url,headers=header,proxies=random.choice(IP_pool)) 
    #print("正在爬取:", url, "状态:",response)
    if  response.status_code == 200:
        ## 解析文档
        root = etree.HTML(response.text) 
        ## 爬文本内容
        temp_content = root.xpath("//div[@id='mainbody']//div[@id='zwconbody']//text()")
        content = text_deal(temp_content)
        ## 爬文本时间
        temp_time = root.xpath("//div[@id='mainbody']//div[@id='zwcontent']//div[@class='zwfbtime']/text()")
        date = text_deal(temp_time)
    else:
        #print("状态错误:", response.status_code)
        content = "ERROR"
        date = "ERROR"

    ## 设置睡眠，反爬虫
    t = random.uniform(0, 1.5)
    time.sleep(t)
    return content, date, response.status_code


In [ ]:
def get_part_list(url_list,part_nums, start=0):  ## 获取 分部分爬虫list
    url_nums = len(url_list)
    temp_index = np.arange(start,url_nums,part_nums).tolist()
    temp_index.append(url_nums)
    
    part_url_list = []
    for i in range(len(temp_index)-1):
        part_url_list.append(url_list[temp_index[i]:temp_index[i+1]])
    return part_url_list

In [ ]:
sleep_time = 6 ## 一个part 6秒
part_nums = 10000   ## 一个part的数量
threading_num = 200 ## 进程数量

## 注： part_nums//threading_num < 99   【5000//60】？ 【10000//120】

In [ ]:
part_url_list = get_part_list(url_list,part_nums, start=0)

In [ ]:
len(part_url_list)

In [ ]:
len(part_url_list[-1])

In [ ]:
"""
#存在错误 Invalid URL '\x1a': No scheme supplied. Perhaps you meant http://? 时，用于检验与处理
"""

len(part_url_list[-1])
for i in range(len(part_url_list[-1])):        
    if part_url_list[-1][i][:8] == "https://":
        pass
    else:
        print(i)
del(part_url_list[-1][-1]) 

In [ ]:
## 生成容器
all_data = []

In [ ]:
run_part_i = int(input("选择运行part的起点："))   ## 目前在【 0 】    ## 选择运行的起点--针对报错停止后重新运行

In [ ]:
start = time.time()
for i,part_url in enumerate(part_url_list):
    if i < run_part_i:   ## 错误位置重新运行
        pass
    else:
        print("*******正在处理第 【", i ,"】 part*******")
        p = ThreadPool(threading_num)
        content = p.map(get_guba_content,part_url)   ## [(content, data, stata)]

        if content[-1][-1] == 200:
            all_data += content
        else:
            print("??????错误在第 【", i ,"】 part???????")
            break

        print("#######正在睡眠#######")
        time.sleep(sleep_time)
end = time.time()

print("运行时间：",(end-start))

In [ ]:
len(all_data)

In [ ]:
df = pd.DataFrame(data=all_data,columns=["content","date","stata"])

In [ ]:
df

In [ ]:
#run_part_i = int(input("保存运行part的终点：")) 

In [ ]:
## 保存 data_content_{stock}_{block}_{run_part_i}.csv
#save_name = f"D:/股票指数类/data/data_content_{stock}_{block}_{run_part_i}.csv"  ## _{start_page}_{end_page}
save_name = f"D:/股票指数类/data/data_content_{stock}_ETF.csv"  ## _{start_page}_{end_page}
df.to_csv(save_name,header=0,index=False) # header=0,

In [ ]:
## 每次保存完记清空内存